# Advanced SQL Queries
<hr style="border:2px solid black">

## 1. Setup

**Load packages**

In [1]:
import sqlalchemy as db
db.__version__

'1.4.47'

In [2]:
import os
import pandas as pd

**connection string ("URL" for database)**

In [10]:
HOST = 'localhost' 
PORT = '5432'
USERNAME = 'wukaiti'
PASSWORD = os.getenv('PG_PASSWORD')
DB = 'bergamot_encoder'

# connection string for Linux
cs_linux = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}"

**connect to database**

In [15]:
# database engine created
engine = db.create_engine(cs_linux, echo=False)
engine = engine.connect()

<hr style="border:2px solid black">

## 2. Sub-Queries
- a subquery is a nested query, i.e., a query within a query
- allows the construction of more complex queries
- needed while executing a query based on the result of another query 

### Example 1
- ***Get a list of the most and least expensive products (name and unit price)***

**query**

In [16]:
query = db.text('''
SELECT product_name, unit_price 
FROM products 
WHERE unit_price = (SELECT MIN(unit_price) FROM products)
OR unit_price = (SELECT MAX(unit_price) FROM products);
''')

**get query result as a dataframe**

In [17]:
pd.read_sql_query(query, engine)

,product_name,unit_price
0,Geitost,2.5
1,Côte de Blaye,263.5


### Example 2
- ***Get all the products of above average price***

**query**

In [25]:
query =db.text( '''
SELECT product_id, product_name, unit_price
FROM products 
WHERE unit_price > (SELECT AVG(unit_price) FROM products); 
''')
#from "WHERE" it is a subquery

**get query result as a dataframe**

In [26]:
pd.read_sql_query(query, engine)

,product_id,product_name,unit_price
0,7,Uncle Bob's Organic Dried Pears,30.00
1,8,Northwoods Cranberry Sauce,40.00
2,9,Mishi Kobe Niku,97.00
3,10,Ikura,31.00
4,12,Queso Manchego La Pastora,38.00
5,17,Alice Mutton,39.00
6,18,Carnarvon Tigers,62.50
7,20,Sir Rodney's Marmalade,81.00
8,26,Gumbär Gummibärchen,31.23
9,27,Schoggi Schokolade,43.90


### Example 3
- ***Calculate the percentage of a product on the total number of orders***

**query**

In [27]:
query = db.text('''
WITH total_quantity AS (SELECT SUM(quantity) FROM order_details)
SELECT product_name, SUM(quantity)*100.0/(TABLE total_quantity) AS percentage 
FROM order_details od
JOIN products p
ON od.product_id = p.product_id
GROUP BY product_name 
ORDER BY percentage DESC;
''')

**get query result as a dataframe**

In [28]:
pd.read_sql_query(query, engine)

,product_name,percentage
0,Camembert Pierrot,3.073056
1,Raclette Courdavault,2.915213
2,Gorgonzola Telino,2.722295
3,Gnocchi di nonna Alice,2.461173
4,Pavlova,2.256562
...,...,...
72,Laughing Lumberjack Lager,0.358556
73,Chocolade,0.268917
74,Gravad lax,0.243584
75,Genen Shouyu,0.237738


### Example 4
- ***Which employees have dealt with very light orders ("freight" value less than 0.5)?***

**query**

In [29]:
query = db.text('''
SELECT first_name, last_name
FROM employees AS e
WHERE EXISTS
(SELECT * FROM orders AS o
WHERE o.employee_id = e.employee_id
AND freight < 0.5);
''')

**get query result as a dataframe**

In [30]:
pd.read_sql_query(query, engine)

,first_name,last_name
0,Nancy,Davolio
1,Andrew,Fuller
2,Janet,Leverling
3,Margaret,Peacock
4,Michael,Suyama
5,Robert,King
6,Laura,Callahan
7,Anne,Dodsworth


<hr style="border:2px solid black">

## 3. String Functions

- helps edit, combine or alter text data columns

### Example 1
- ***Get the full name, and create email addreses of the employees***

**query**

In [38]:
query = db.text(f'''
SELECT (first_name || ' ' || last_name) AS full_name,
(lower(first_name) || '.' || lower(last_name) || '@northwind.com') AS email_address
FROM employees;
''')

**get query result as a dataframe**

In [39]:
pd.read_sql_query(query, engine)

,full_name,email_address
0,Nancy Davolio,nancy.davolio@northwind.com
1,Andrew Fuller,andrew.fuller@northwind.com
2,Janet Leverling,janet.leverling@northwind.com
3,Margaret Peacock,margaret.peacock@northwind.com
4,Steven Buchanan,steven.buchanan@northwind.com
5,Michael Suyama,michael.suyama@northwind.com
6,Robert King,robert.king@northwind.com
7,Laura Callahan,laura.callahan@northwind.com
8,Anne Dodsworth,anne.dodsworth@northwind.com


### Example 2
- ***Maximum number of characters in employee last name***

**query**

In [40]:
query = db.text(f'''
SELECT MAX(LENGTH(last_name)) 
FROM employees;
''')

**get query result**

In [41]:
engine.execute(query).fetchall()[0][0]

9

<hr style="border:2px solid black">

## 4. Self Joins

- a query in which a table gets joined to itself
- useful for comparing values in a column of different rows in a table

### Example 1
- ***Show the employee names along with their reports' recipient names***

**query**

In [42]:
query = db.text(f'''
SELECT emp.last_name AS employee, rep.last_name AS report_recepient
FROM employees AS emp
JOIN employees AS rep
ON rep.employee_id = emp.reportsto;
''')

**get query result as a dataframe**

In [43]:
pd.read_sql_query(query, engine)

,employee,report_recepient
0,Davolio,Fuller
1,Leverling,Fuller
2,Peacock,Fuller
3,Buchanan,Fuller
4,Suyama,Buchanan
5,King,Buchanan
6,Callahan,Fuller
7,Dodsworth,Buchanan


### Example 2
- ***Find all pairs of products supplied by Exotic Liquids***

**query**

In [44]:
query = db.text(f'''
SELECT p1.product_name, p2.product_name
FROM products AS p1
JOIN products AS p2
ON p1.product_id != p2.product_id
AND p1.supplier_id = p2.supplier_id
JOIN suppliers AS s
ON s.supplier_id = p1.supplier_id
WHERE s.company_name = 'Exotic Liquids';
''')

**get query result as a dataframe**

In [45]:
pd.read_sql_query(query, engine)

,product_name,product_name
0,Aniseed Syrup,Chai
1,Chang,Chai
2,Aniseed Syrup,Chang
3,Chai,Chang
4,Chang,Aniseed Syrup
5,Chai,Aniseed Syrup


<hr style="border:2px solid black">

## 5. Indexing

- indices are lookup data structures that accelerate your database queries
- queries run faster only if the indexed column is used in a query
- indices trade memory for computation time, so the total size of your database on the disk grows

### Index Creation

**single-column index**

`CREATE INDEX table_name_column_name_idx
ON table_name(column_name);`

**multi-column index**

`CREATE INDEX table_name_column1_column2_idx
ON table_name(column2, column1);`

<hr style="border:2px solid black">

## References

- [Advanced SQL for Data Scientists](https://medium.com/analytics-vidhya/advanced-sql-for-data-scientists-8e99a3c7f24c)
- [7 Advanced SQL Concepts You Need to Know!](https://medium.com/dp6-us-blog/7-advanced-sql-concepts-you-need-to-know-45fa149ba0b0)
- [SQL Index: YouTube video by Socratica](https://www.youtube.com/watch?v=fsG1XaZEa78)